In [1]:
from z3 import *

In [5]:

def get_element_at_index(arr, idx):
    # Start with the first element as the default
    selected = arr[0]
    
    # Iterate through the list, select the element where the index matches
    for i in range(1, len(arr)):
        selected = If(idx == i, arr[i], selected)
    
    return selected

def Min(lst):
    # Start with the first element as the minimum.
    min_value = lst[0]
    
    # Iterate through the list, updating min_value with the smallest element found.
    for elem in lst[1:]:
        min_value = If(elem < min_value, elem, min_value)
    
    return min_value

def subcircuit_chatgpt(x):
    S = range(len(x))  # index_set(x)
    l = min(S)
    u = max(S)
    n = len(S)

    print(f"S = {S}")
    print(f"l = {l}")
    print(f"u = {u}")
    print(f"n = {n}")
    # Variables
    order = IntVector('order', n)
    ins = [Bool(f'ins_{i}') for i in S]
    for i in S:
        ins[i] = (x[i] != i + 1)

    firstin = Int('firstin')
    lastin = Int('lastin')
    empty = Bool('empty')

    constraints = []

    # Constraints for order and firstin
    constraints.append(Distinct(x))
    constraints.append(Distinct(order))
    constraints.append(firstin == Min([If(ins[i], i, u+1) for i in S]))
    constraints.append(empty == (firstin > u))

    # If the subcircuit is empty, then each node points at itself.
    for i in S:
        constraints.append(Implies(empty, Not(ins[i])))

    # If the subcircuit is non-empty, then order numbers the subcircuit.
    non_empty_conditions = []

    # The firstin node is numbered 1.
    non_empty_conditions.append(get_element_at_index(order, firstin) == 1)

    # The lastin node is greater than firstin.
    non_empty_conditions.append(lastin > firstin)

    # The lastin node points at firstin.
    #non_empty_conditions.append(x[lastin] == firstin)
    non_empty_conditions.append(get_element_at_index(x, lastin) == firstin + 1)

    # And both are in
    #non_empty_conditions.append(ins[lastin])
    #non_empty_conditions.append(ins[firstin])
    non_empty_conditions.append(get_element_at_index(ins, lastin))
    non_empty_conditions.append(get_element_at_index(ins, firstin))

    # The successor of each node except where it is firstin is
    # numbered one more than the predecessor.
    for i in S:
        non_empty_conditions.append(Implies(And(ins[i], x[i] - 1 != firstin), 
                                             get_element_at_index(order, x[i] - 1) == get_element_at_index(order, i) + 1))

    # Each node that is not in is numbered after the lastin node.
    for i in S:
        non_empty_conditions.append(Or(ins[i], get_element_at_index(order, lastin) < get_element_at_index(order, i)))

    constraints.append(Implies(Not(empty), And(non_empty_conditions)))

    return constraints


In [21]:
solver = Solver()
my_paths = [[3, 2, 4, 7, 5, 6, 1], [1, 5, 3, 4, 6, 7, 2]]

my_paths = my_paths[1:2]

PATH = [[Int(f"path_{i}_{j}") for j in range(len(my_paths[0]))] for i in range(len(my_paths))]
for i in range(len(my_paths)):
    for j in range(len(my_paths[0])):
        solver.add(PATH[i][j] == my_paths[i][j])

# Check if the path is a subcircuit
for i in range(len(my_paths)):
    solver.add(subcircuit_chatgpt(PATH[i]))

print(solver.check())
model = solver.model()
# Print the path
print(f"PATH = {[model[PATH[i][j]].as_long() for i in range(len(my_paths)) for j in range(len(my_paths[0]))]}")

S = range(0, 7)
l = 0
u = 6
n = 7
sat
PATH = [1, 5, 3, 4, 6, 7, 2]
